In [1]:
import pandas as pd
import numpy as np
import re
import emoji
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS,WordCloud,ImageColorGenerator

In [2]:
# Extract Date
def date_time(s):
    pattern = "^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -"
    result = re.match(pattern,s)
    if result:
        return True
    else:
        return False


In [3]:
print(date_time("29/06/2021, 11:18 am -"))


True


In [4]:
# Find contact

def find_contact(s):
    s = s.split(":")
    if len(s) == 2:
        return True
    else:
        return False

In [5]:
print(find_contact("Rahul Sir: <Media omitted>"))


True


In [6]:
# Finding MSG

def find_msg(line):
    splitline = line.split(" - ")
    dateTime = splitline[0]
    date, time = dateTime.split(", ")
    message = " ".join(splitline[1:])

    if find_contact(message):
        splitmessage = message.split(": ")
        contact = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        contact = None
        
    return date, time, contact, message

In [7]:
data = []
txt_file = r"D:\PFP\Projects\WhatsApp Sentiment Analysis\WhatsApp Chat with LEC IT 7th sem.txt"

with open(txt_file,encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, contact = None,None,None

    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([date,time,contact," ".join(messageBuffer)])
            
            messageBuffer.clear()
            date,time,contact,message = find_msg(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)


In [8]:
df = pd.DataFrame(data, columns=["Date", 'Time', 'Contact Name', 'Message'])
df


,Date,Time,Contact Name,Message
0,19/08/2019,4:40 pm,None,"Rakesh Sir IT Morbi created group ""LEC IT 3rd ..."
1,19/08/2019,4:40 pm,None,+91 76220 05315 was added
2,13/11/2019,12:54 pm,Rahul Sir,<Media omitted>
3,13/11/2019,12:54 pm,Rahul Sir,<Media omitted>
4,13/11/2019,12:54 pm,Rahul Sir,<Media omitted>
...,...,...,...,...
2300,10/07/2021,7:03 pm,None,Rakesh Sir IT Morbi: https://forms.office.com/...
2301,10/07/2021,7:03 pm,Rakesh Sir IT Morbi,Microsoft Tech Saksham Program for Girls Stude...
2302,12/07/2021,11:31 am,Hema Ma'am,Join for spm
2303,12/07/2021,11:31 am,Hema Ma'am,Fast


In [9]:
df.isnull().sum()

Date              0
Time              0
Contact Name    436
Message           0
dtype: int64

In [10]:
df.dropna(inplace=True)

In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sentiments = SentimentIntensityAnalyzer()

df["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in df["Message"]]
df["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in df["Message"]]
df["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in df["Message"]]


In [12]:
df

,Date,Time,Contact Name,Message,Positive,Negative,Neutral
2,13/11/2019,12:54 pm,Rahul Sir,<Media omitted>,0.000,0.0,1.000
3,13/11/2019,12:54 pm,Rahul Sir,<Media omitted>,0.000,0.0,1.000
4,13/11/2019,12:54 pm,Rahul Sir,<Media omitted>,0.000,0.0,1.000
5,13/11/2019,12:55 pm,Rahul Sir,Pending approved applications,0.583,0.0,0.417
6,13/11/2019,12:55 pm,Rahul Sir,Also Remaining all applications have been appr...,0.318,0.0,0.682
...,...,...,...,...,...,...,...
2299,09/07/2021,3:34 pm,Tejas Sir,Join for PR now,0.423,0.0,0.577
2301,10/07/2021,7:03 pm,Rakesh Sir IT Morbi,Microsoft Tech Saksham Program for Girls Stude...,0.081,0.0,0.919
2302,12/07/2021,11:31 am,Hema Ma'am,Join for spm,0.524,0.0,0.476
2303,12/07/2021,11:31 am,Hema Ma'am,Fast,0.000,0.0,1.000


In [13]:
x = sum(df["Positive"])
y = sum(df["Negative"])
z = sum(df["Neutral"])

In [14]:
print(x,y,z)

250.12499999999977 59.262 1554.610999999998


In [15]:
def sentiment_score(a,b,c):
    if (a>b) and (b>c):
        print("Positive 😊")
    elif (b>a) and (b>c):
        print("Negative 😠")
    else:
        print("Neutral 🙂")

sentiment_score(x,y,z)


Neutral 🙂
